In [0]:
%sql
use catalog dev_edh;
use schema dummy;

In [0]:
df = spark.table('used_car_sales_bronze')

In [0]:
from pyspark.sql.functions import col, when,length

cleaned_df =     df \
                .dropna(subset=['pricesold', 'yearsold', 'Mileage', 'Make', 'Model', 'Year', 'zipcode']) \
                .withColumn("Age", col("yearsold").cast('integer') - col("Year").cast('integer')) \
                .withColumn("PricePerMile", col("pricesold") / col("Mileage")) \
                .withColumn("Year", col("Year").cast('string').substr(1,4) )

# categorize mileage bracket based on mileage
cleaned_df = cleaned_df \
     .withColumn("MileageBracket", \
     when(col("Mileage") < 20000, "0-20000") \
    .when((col("Mileage") >= 20000) & (col("Mileage") < 40000), "20000-40000") \
    .when((col("Mileage") >= 40000) & (col("Mileage") < 60000), "40000-60000") \
    .when((col("Mileage") >= 60000) & (col("Mileage") < 80000), "60000-80000")  \
    .when((col("Mileage") >= 80000) & (col("Mileage") < 100000), "80000-100000") \
    .when((col("Mileage") >= 100000) & (col("Mileage") < 150000), "100000-150000") \
    .when((col("Mileage") >= 150000), "150000+")) \
    .withColumn("PriceBracket", \
     when(col("pricesold") < 5000, "0-5000 ") \
    .when((col("pricesold") >= 5000) & (col("pricesold") < 10000), "5000-10000 ") \
    .when((col("pricesold") >= 10000) & (col("pricesold") < 15000), "10000-15000  ") \
    .when((col("pricesold") >= 15000) & (col("pricesold") < 20000), "15000-20000  ") \
    .when((col("pricesold") >= 20000) & (col("pricesold") < 25000), "20000-25000 ")  \
    .when((col("pricesold") >= 25000) & (col("pricesold") < 30000), "25000-30000 ") \
    .when((col("pricesold") >= 30000) & (col("pricesold") < 35000), "30000-35000 ") \
    .when((col("pricesold") >= 35000) & (col("pricesold") < 40000), "35000-40000 ") \
    .when((col("pricesold") >= 40000) & (col("pricesold") < 45000), "40000-45000 ") \
    .when((col("pricesold") >= 45000) & (col("pricesold") < 50000), "45000-50000 ") \
    .when((col("pricesold") >= 50000) & (col("pricesold") < 75000), "55000-75000 ") \
    .when((col("pricesold") >= 75000) & (col("pricesold") < 100000), "75000-100000 ") \
    .when((col("pricesold") >= 100000) , "1000000+"))

display(cleaned_df)

In [0]:
from pyspark.sql.functions import length

cleaned_df =  (
               cleaned_df \
                         .filter((col('pricesold') > 100) & (col('Year').cast('int') >= 1940) & (col('Year').cast('int') <= 2025))
                         .filter(col('Age') >= 0)
                         .filter(length(col('zipcode')) >= 5)
                )         
display(cleaned_df)

In [0]:
cleaned_df.write.mode("overwrite").saveAsTable("car_sales_silver")

In [0]:
%sql
select * from car_sales_silver